<a href="https://colab.research.google.com/github/Mark-THU/load_forecast/blob/main/CNN_24_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip3 install xgboost --upgrade
# Using CNN for prediction of 24 hours
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
import torch
import torch.nn as nn

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sqlalchemy import create_engine
from xgboost import XGBRegressor
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"
device = torch.device(dev)

In [3]:
# load data
# url = 'https://raw.githubusercontent.com/Mark-THU/load_forecast/main/integrate_0101510000.csv'
url = 'https://raw.githubusercontent.com/Mark-THU/load_forecast/main/dataset.csv'
data = pd.read_csv(url, sep='\t', index_col='time')
# data = data[['tembody','tem', 'is_holiday', 'day_of_week', 'load']]

In [4]:
# 归一化
def normalization(data):
    """
    data: original data with load
    return: normalized data, scaler of load
    """
    normalized_data = MinMaxScaler().fit_transform(data)
    scaler_y = MinMaxScaler()
    scaler_y.fit_transform(data[['load']])
    return normalized_data, scaler_y

In [5]:
# build supervised data
def Series_To_Supervise(data, seq_len, y_col_index):
    """
    convert series data to supervised data
    :param data: original data
    :param seq_len: length of sequence
    :y_col_index: index of column which acts as output
    :return: return two ndarrays-- input and output in format suitable to feed to LSTM
    """
    dim_0 = data.shape[0] - seq_len
    dim_1 = data.shape[1]
    x = np.zeros((dim_0, seq_len, dim_1))
    y = np.zeros((dim_0,))
    for i in range(dim_0):
        x[i] = data[i: i+seq_len]
        y[i] = data[i+seq_len, y_col_index]
    print("shape of x: {}, shape of y: {}".format(x.shape, y.shape))
    return x, y

In [6]:
# 5-fold cross-validation
def split_dataset(X, Y, n_split=5):
    """
    X: original feature, size * 72 * features
    Y: labels, size * 1
    return: list of train_x, test_x, train_y, test_y
    """
    kf = KFold(n_splits=n_split, shuffle=True, random_state=1)
    train_x_list = list()
    valid_x_list = list()
    test_x_list = list()
    train_y_list = list()
    valid_y_list = list()
    test_y_list = list()
    for train_index, test_index in kf.split(X):
        train_x_list.append(X[train_index])
        train_y_list.append(Y[train_index])
        test_x = X[test_index]
        test_y = Y[test_index]
        valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=1)
        valid_x_list.append(valid_x)
        valid_y_list.append(valid_y)
        test_x_list.append(test_x)
        test_y_list.append(test_y)
    return train_x_list, valid_x_list, test_x_list, train_y_list, valid_y_list, test_y_list

In [7]:
# define CNN model 
class CNN(nn.Module):
    """
    A CNN neural network
    """
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=13, out_channels=26, kernel_size=3, groups=13),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3),
            nn.Flatten(),
            nn.Linear(26*15, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
        )
    
    def forward(self, x):
        out = self.cnn(x)
        out = out[:, -1]
        return out

In [8]:
# train the model 
def Train_Model(train_x, train_y, valid_x, valid_y, batch_size, lr, number_epoch, seq_len):
#     pdb.set_trace()
    model_cnn = CNN()
    model_cnn = model_cnn.to(device)
    # transpose train_x, valid_x
    train_x = train_x[:, 0:seq_len].transpose(0, 2, 1)
    valid_x = valid_x[:, 0:seq_len].transpose(0, 2, 1)
    train_dataset = TensorDataset(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
    valid_dataset = TensorDataset(torch.FloatTensor(valid_x), torch.FloatTensor(valid_y))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model_cnn.parameters(), lr=lr)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    valid_loss_min = np.Inf
    num_without_imp = 0
    for epoch in range(1, number_epoch+1):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)            
            optimizer.zero_grad()
            outputs = model_cnn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
#                 if num_without_imp > 20:
#                     return model_cnn
                num_without_imp = num_without_imp + 1
                valid_losses = list()
                model_cnn.eval()
                for inp, lab in valid_loader:
                    inp = inp.to(device)
                    lab = lab.to(device)
                    # pdb.set_trace()
                    out = model_cnn(inp)
                    valid_loss = criterion(out, lab)
                    valid_losses.append(valid_loss.item())
                
                model_cnn.train()
                print("Epoch: {}/{}...".format(epoch, number_epoch),
                     "Step: {}/{}...".format(i+1, len(train_dataset)//batch_size),
                     "Loss: {:.8f}...".format(loss.item()),
                     "Valid Loss: {:.8f}...".format(np.mean(valid_losses)))
                if np.mean(valid_losses) < valid_loss_min:
                    num_without_imp = 0
                    torch.save(model_cnn.state_dict(), "cnn_state_dict.pt")
                    print("Valid loss decreased ({:.6f}-->{:.6f}). Saving model...".format(valid_loss_min, np.mean(valid_losses)))
                    valid_loss_min = np.mean(valid_losses)
    return model_cnn

In [9]:
# test model by hour
def Test_Model(model, test_x, test_y, batch_size, seq_len, scaler):
    # transpose test_x
    test_x = test_x[:, 0:seq_len].transpose(0, 2, 1)
    test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))
    test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    model.load_state_dict(torch.load('cnn_state_dict.pt'))
    y_pred = list()
    y_true = list()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            y_pred = y_pred + outputs.cpu().numpy().tolist()
            y_true = y_true + labels.cpu().numpy().tolist()
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_pred = y_pred.reshape(-1, 1)
    y_true = y_true.reshape(-1, 1)
    load_pred = scaler.inverse_transform(y_pred)
    load_true = scaler.inverse_transform(y_true)
    MAPE = np.mean(np.abs(load_true-load_pred)/load_true)
    return MAPE

In [10]:
# test model by day
def Test_Model_24(model, test_x, test_y, batch_size, seq_len, y_col_index, scaler):
    # transpose test_x
    test_x = test_x.transpose(0, 2, 1)
    test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    model.load_state_dict(torch.load('cnn_state_dict.pt'))
    size = 24
    y_pred = list()
    y_true = list()
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            for i in range(size):
#                 pdb.set_trace()
                inp = inputs[:, :, i:i+seq_len]
                out = model(inp)
                y_pred = y_pred + out.cpu().numpy().tolist()
                y_true = y_true + inputs[:, y_col_index, i+seq_len].cpu().numpy().tolist()
                inputs[:, y_col_index, i+seq_len] = out
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_pred = y_pred.reshape(-1, 1)
    y_true = y_true.reshape(-1, 1)    
    load_pred = scaler.inverse_transform(y_pred)
    load_true = scaler.inverse_transform(y_true)
    MAPE = np.mean(np.abs(load_true-load_pred)/load_true)
    return MAPE

In [11]:
# model configs
def Model_Configs():
    batch_sizes = [512]
    lrs = [0.01]
    number_epochs = [80]
    configs = list()
    for i in batch_sizes:
        for j in lrs:
            for k in number_epochs:
                    configs.append([i, j ,k])
    return configs

In [12]:
def main(n_split=5, seq_len=48):
    # 归一化
    normalized_data, scaler_y = normalization(data)
    # 数据处理，将时间序列数据变为监督学习数据
    y_index = normalized_data.shape[1] -1
    _, Y = Series_To_Supervise(normalized_data, seq_len=seq_len, y_col_index=y_index) 
    # 五折交叉验证，划分数据集
    # 为了方便测试24点准确性， seq_len需要+24
    X, _ = Series_To_Supervise(normalized_data, seq_len=seq_len + 24, y_col_index=y_index)
    Y = Y[:-24]
    train_x_list, valid_x_list, test_x_list, train_y_list, valid_y_list, test_y_list = split_dataset(X, Y)
    print("model configs set")
    configs = Model_Configs()
    MAPE_list = list()
    MAPE_list_24 = list()
    for config in configs:
        batch_size = config[0]
        lr = config[1]
        number_epoch = config[2]
        print("Config: batch_size--{}, lr--{}, number_epoch--{}".format(config[0], config[1], config[2]))
        tmp_list = list()
        tmp_list_24 = list()
        for i in range(n_split):
            while(1):
                model = Train_Model(train_x_list[i], train_y_list[i], valid_x_list[i], valid_y_list[i], batch_size, lr, number_epoch=number_epoch,
                                    seq_len=seq_len)
                MAPE = Test_Model(model, test_x_list[i], test_y_list[i], batch_size, seq_len, scaler=scaler_y)
                MAPE_24 = Test_Model_24(model, test_x_list[i], test_y_list[i], batch_size, seq_len,
                                        y_col_index=y_index, scaler=scaler_y)
                if MAPE < 0.1:
                    break
            tmp_list.append(MAPE)
            tmp_list_24.append(MAPE_24)
        MAPE_list.append(tmp_list)
        MAPE_list_24.append(tmp_list_24)
    return (MAPE_list, MAPE_list_24)

In [ ]:
(MAPE_list, MAPE_list_24) = main(n_split=5)

shape of x: (22769, 48, 13), shape of y: (22769,)
shape of x: (22745, 72, 13), shape of y: (22745,)
model configs set
Config: batch_size--512, lr--0.01, number_epoch--80
Epoch: 1/80... Step: 1/35... Loss: 0.22431317... Valid Loss: 0.78092319...
Valid loss decreased (inf-->0.780923). Saving model...
Epoch: 1/80... Step: 11/35... Loss: 0.02439280... Valid Loss: 0.03883083...
Valid loss decreased (0.780923-->0.038831). Saving model...
Epoch: 1/80... Step: 21/35... Loss: 0.02307430... Valid Loss: 0.02461026...
Valid loss decreased (0.038831-->0.024610). Saving model...
Epoch: 1/80... Step: 31/35... Loss: 0.01721098... Valid Loss: 0.01812807...
Valid loss decreased (0.024610-->0.018128). Saving model...
Epoch: 2/80... Step: 1/35... Loss: 0.01538365... Valid Loss: 0.01328465...
Valid loss decreased (0.018128-->0.013285). Saving model...
Epoch: 2/80... Step: 11/35... Loss: 0.00933376... Valid Loss: 0.00903458...
Valid loss decreased (0.013285-->0.009035). Saving model...
Epoch: 2/80... Step: 

In [ ]:
print("MAPE: {:.6f}".format(np.array(MAPE_list).mean()))
print("MAPE_24: {:.6f}".format(np.array(MAPE_list_24).mean()))